In [66]:
import torch
from torch import nn
from torch.nn import functional as F

In [67]:
net = nn.Sequential(nn.Linear(20, 256),nn.ReLU(),nn.Linear(256, 10))

X = torch.rand(2, 20)

O = net(X)

print(O)

tensor([[-0.1324, -0.0161, -0.3142,  0.2156, -0.0647,  0.0020, -0.0979,  0.1059,
          0.0108,  0.0246],
        [-0.1463,  0.0578, -0.1957,  0.4009, -0.2304,  0.0741, -0.0859,  0.0781,
          0.0261, -0.1042]], grad_fn=<AddmmBackward>)


In [68]:
class MLP(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

net = MLP()
net(X)

tensor([[-0.1314, -0.1541, -0.0966, -0.2746,  0.2238,  0.1905,  0.0981, -0.1696,
          0.0571,  0.0527],
        [-0.0901, -0.1874, -0.2177, -0.3595,  0.1585,  0.0702,  0.1541, -0.2932,
         -0.0799,  0.0661]], grad_fn=<AddmmBackward>)

In [69]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            self._modules[block] = block

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.0380, -0.0865,  0.0322,  0.4502, -0.0387, -0.4043, -0.0417,  0.2229,
          0.1033, -0.1013],
        [ 0.0133, -0.0633,  0.0677,  0.4148,  0.0981, -0.4065, -0.0681,  0.2187,
          0.2793, -0.0486]], grad_fn=<AddmmBackward>)

In [70]:
class FixedMLP(nn.Module):

    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)

        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedMLP()
net(X)

tensor(0.3785, grad_fn=<SumBackward0>)

In [71]:
class NestMLP(nn.Module):

    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedMLP())
chimera(X)
print(chimera)


Sequential(
  (0): NestMLP(
    (net): Sequential(
      (0): Linear(in_features=20, out_features=64, bias=True)
      (1): ReLU()
      (2): Linear(in_features=64, out_features=32, bias=True)
      (3): ReLU()
    )
    (linear): Linear(in_features=32, out_features=16, bias=True)
  )
  (1): Linear(in_features=16, out_features=20, bias=True)
  (2): FixedMLP(
    (linear): Linear(in_features=20, out_features=20, bias=True)
  )
)
